---
title: Qiskit Runtime local testing mode
description: Use local testing mode to simulate quantum hardware in Qiskit Runtime
---


{/* cspell:ignore superop */}

<span id="test-locally"></span>
# Qiskit Runtime local testing mode

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
qiskit-aer~=0.17
```
</details>

Use local testing mode (available with `qiskit-ibm-runtime` v0.22.0 or later) to test programs before fine-tuning them and sending them to real quantum hardware.  After using local testing mode to verify your program, all you need to change is the backend name to run it on a QPU.

To use local testing mode, specify one of the fake backends from ``qiskit_ibm_runtime.fake_provider`` or specify a Qiskit Aer backend when instantiating a Qiskit Runtime primitive or a session.

- **Fake backends**: The [fake backends](/docs/api/qiskit-ibm-runtime/fake-provider)  in ``qiskit_ibm_runtime.fake_provider`` mimic the behaviors of IBM&reg; QPUs by using QPU snapshots. The QPU snapshots contain important information about the QPU, such as the coupling map, basis gates, and qubit properties, which are useful for testing the transpiler and performing noisy simulations of the QPU. The noise model from the snapshot is automatically applied during simulation.
- **Aer simulator**: Simulators from [Qiskit Aer](/docs/guides/simulate-with-qiskit-aer) provide higher-performance simulation that can handle larger circuits and [custom noise models](/docs/guides/build-noise-models). A list of simulation method options are available when you use `AerSimulator` in local testing mode. See the [Clifford simulation mode example](#clifford-sim), which demonstrates how to efficiently simulate Clifford circuits with a large number of qubits.

    <details>
    <summary>**List of simulation methods available from Qiskit Aer**</summary>

    See the [`AerSimulator`](https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator) documentation for more information.

    * ``"automatic"``: Default simulation method. Select the simulation method automatically, based on the circuit and noise model.

    * ``"statevector"``: A dense statevector simulation that can sample measurement outcomes from *ideal* circuits with all measurements at the end of the circuit. For noisy simulations, each shot samples a randomly sampled noisy circuit from the noise model.

    * ``"density_matrix"``: A density matrix simulation that can sample measurement outcomes from *noisy* circuits with all measurements at the end of the circuit.

    * ``"stabilizer"``: An efficient Clifford stabilizer state simulator that can simulate noisy Clifford circuits if all errors in the noise model are also Clifford errors.

    * ``"extended_stabilizer"``: An approximate simulator for Clifford + T circuits based on decomposing the state into a ranked-stabilizer state. The number of terms grows with the number of non-Clifford (T) gates.

    * ``"matrix_product_state"``: A tensor-network statevector simulator that uses a Matrix Product State (MPS) representation for the state. This can be done with or without truncating the MPS bond dimensions, depending on the simulator options. The default behavior is no truncation.

    * ``"unitary"``: A dense unitary matrix simulation of an ideal circuit. This simulates the unitary matrix of the circuit itself, rather than the evolution of an initial quantum state. This method can only simulate gates; it does not support measurement, reset, or noise.

    * ``"superop"``: A dense superoperator matrix simulation of an ideal or noisy circuit. This simulates the superoperator matrix of the circuit itself, rather than the evolution of an initial quantum state. This method can simulate ideal and noisy gates and resets, but it does not support measurements.

    * ``"tensor_network"``: A tensor-network-based simulation that supports both statevector and density matrix. Currently this is only available for GPU and is accelerated by using cuQuantum `cuTensorNet` APIs.
    </details>


<Admonition type="note" title="Notes">
- You can specify all Qiskit Runtime options in local testing mode.  However, all options except shots are ignored when run on a local simulator.
- It is recommended that you install Qiskit Aer before using fake backends or Aer simulators by running `pip install qiskit-aer`. The fake backends use Aer simulators under the cover if available, to take advantage of their performance.
</Admonition>


{/* If you currently use the Qiskit SDK, you can use the existing Qiskit Aer primitives for local testing.  See [exact and noisy simulation with Qiskit Aer primitives](/docs/guides/simulate-with-qiskit-aer) for details. */}

## Fake backends example

In [1]:
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# Bell Circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Run the sampler job locally using FakeManilaV2
fake_manila = FakeManilaV2()
pm = generate_preset_pass_manager(backend=fake_manila, optimization_level=1)
isa_qc = pm.run(qc)

# You can use a fixed seed to get fixed results.
options = {"simulator": {"seed_simulator": 42}}
sampler = Sampler(mode=fake_manila, options=options)

result = sampler.run([isa_qc]).result()

## AerSimulator examples

Example with sessions, without noise:

In [2]:
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler

# Bell Circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_qc = pm.run(qc)
with Session(backend=aer_sim) as session:
    sampler = Sampler(mode=session)
    result = sampler.run([isa_qc]).result()

To simulate with noise, specify a QPU (quantum hardware) and submit it to Aer.  Aer builds a noise model based on the calibration data from that QPU, and instantiates an Aer backend with that model.  If you prefer, you can [build a noise model](/docs/guides/build-noise-models).

<Admonition type="caution">
A QPU can be affected by different kinds of noise. The Qiskit Aer noise model used here only simulates some of them and therefore is likely to be less severe than the noise on a real QPU.

For details on what errors are included when initializing a noise model from a QPU, see the Aer [`NoiseModel`](https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.noise.NoiseModel.html#qiskit_aer.noise.NoiseModel.from_backend) API reference.
</Admonition>

Example with noise:

In [3]:
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService

# Bell Circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

service = QiskitRuntimeService()

# Specify a QPU to use for the noise model
real_backend = service.backend("ibm_fez")
aer = AerSimulator.from_backend(real_backend)

# Run the sampler job locally using AerSimulator.
pm = generate_preset_pass_manager(backend=aer, optimization_level=1)
isa_qc = pm.run(qc)
sampler = Sampler(mode=aer)
result = sampler.run([isa_qc]).result()

<span id="clifford-sim"></span>
### Clifford simulation

Because Clifford circuits can be simulated efficiently with verifiable results, Clifford simulation is a very useful tool. For an in-depth example, see [Efficient simulation of stabilizer circuits with Qiskit Aer primitives](/docs/guides/simulate-stabilizer-circuits).

Example:

In [4]:
import numpy as np
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_runtime import SamplerV2 as Sampler

n_qubits = 500  # <---- note this uses 500 qubits!
circuit = efficient_su2(n_qubits)
circuit.measure_all()

rng = np.random.default_rng(1234)
params = rng.choice(
    [0, np.pi / 2, np.pi, 3 * np.pi / 2],
    size=circuit.num_parameters,
)

# Tell Aer to use the stabilizer (Clifford) simulation method
aer_sim = AerSimulator(method="stabilizer")

pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_qc = pm.run(qc)
sampler = Sampler(mode=aer_sim)
result = sampler.run([isa_qc]).result()

## Next steps

<Admonition type="tip" title="Recommendations">
    - Review detailed [primitives examples.](/docs/guides/primitives-examples)
    - Read [Migrate to V2 primitives](/docs/guides/v2-primitives).
    - Practice with primitives by working through the [Cost function lesson](/learning/courses/variational-algorithm-design/cost-functions) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](/docs/guides/transpile) section.
    - Try the [Compare transpiler settings](/docs/guides/circuit-transpilation-settings#compare-transpiler-settings) tutorial.
</Admonition>